In [1]:
import skimage.io as io
import numpy as np
from skimage.filters import threshold_otsu 
from skimage.color import rgb2grey
from skimage.measure import find_contours
import cv2
from windows import getClusteredWindows
from PolygonFeatures import *
from WindowFeatures import *
import time

In [2]:
def First_difference(chain_code):
    #chain =chain_code[-1] +chain_code
    chain = chain_code
    chain = list(chain)
    chain_shifted = chain[1:]
    chain = chain[:-1]
    chain = np.array(chain).astype(np.int)
    chain_shifted = np.array(chain_shifted).astype(np.int)
    first_diff = -1*(chain - chain_shifted)%8
    first_diff = first_diff.astype(np.str)
    return "".join(first_diff)
    

In [59]:
def First_difference_VEC(chain_code):
    #chain =chain_code[-1] +chain_code

    chain_shifted = chain_code[1:]
    chain = chain_code[:-1]
    first_diff = -1*(chain - chain_shifted)%8
    
    return first_diff
    

In [5]:
def Chain_Codes_Counting(contours,pairs_triplets=True):
    global_chaining_code = ""
    first_difference = ""
    second_difference = ""
    if(pairs_triplets):
        F4 = np.zeros(64)
        F5 = np.zeros(512)
    
    for j in range(len(contours)):
        c = contours[j]
        local_chaining_code = ""
        for i in range(1,len(c)):
            diff = (c[i]-c[i-1])[0]
            if(diff[0]==1 and diff[1]==0):
                local_chaining_code +="0"
            elif(diff[0]==1 and diff[1]==-1):
                local_chaining_code+="1"
            elif(diff[0]==0 and diff[1]==-1):
                local_chaining_code +="2"
            elif(diff[0]==-1 and diff[1]==-1):
                local_chaining_code+="3"
            elif(diff[0]==-1 and diff[1]==0):
                local_chaining_code+="4"
            elif(diff[0]==-1 and diff[1]==1):
                local_chaining_code +="5"
            elif(diff[0]==0 and diff[1]==1):
                local_chaining_code+="6"
            elif(diff[0]==1 and diff[1]==1):
                local_chaining_code+="7"
            else:
                print("ERROR")

        if(len(local_chaining_code)>0):
            first_diff = First_difference(local_chaining_code)
            first_difference += first_diff    
            second_difference += First_difference(first_diff)
        global_chaining_code+=local_chaining_code
      
        if pairs_triplets:
            for  i in range(8):
                for j in range(8):
                    check = str(i)+str(j)
                    F4[i*8+j] += local_chaining_code.count(check)
                    for k in range(8):
                        check = str(i)+str(j)+str(k)
                        F5[i*64+j*8+k] += local_chaining_code.count(check)
    F1  = []
    F2  = []
    F3  = []
    for i in range(8):
        F1.append(global_chaining_code.count(str(i)))
        if i!=5:
            F2.append(first_difference.count(str(i)))

        F3.append(second_difference.count(str(i)))

    F1 = np.array(F1)
    F2 = np.array(F2)
    F3 = np.array(F3)
    F1 = F1/F1.sum()
    F2 = F2/F2.sum()
    F3 = F3/F3.sum()
    if pairs_triplets:
        F4 = F4/F4.sum()
        F5 = F5/F5.sum()
        return F1,F2,F3,F4,F5
    return F1,F2,F3

In [85]:
def Chain_Codes_Counting_VEC(contours,pairs_triplets=True):
    directions = [[1,0],[1,-1],[0,-1],[-1,-1],[-1,0],[-1,1],[0,1],[1,1]]
    global_chain_code =list()#= np.array([])
    first_difference = list()#np.array([])
    second_difference = list()#np.array([])
    p_t = list()
    pairs_tri = np.array([])
    for cont in contours:
        cont = np.squeeze(cont,1)
        cont_diff= cont [1:]-cont[:-1]
        chain_code = np.zeros(len(cont_diff),dtype=np.int)
        for i in range(len(directions)):
            t = (cont_diff==directions[i])
            chain_code[t[:,0] & t[:,1]]=i
        if(len(chain_code)>0):
            first_diff = First_difference_VEC(chain_code)
            first_difference += list(first_diff)#= np.append(first_diff,first_difference)    
            second_difference += list(First_difference_VEC(first_diff)) #= np.append(First_difference_VEC(first_diff),second_difference)
        global_chain_code+=list(chain_code) #= np.append(chain_code,global_chain_code)
        if(pairs_triplets):
            p_t += list(chain_code)
            p_t += [9]#To Divide between two chain codes so i can count the pairs , triplets directly from string
                      #Must be int and not negative and not in range(0--7) because it will change count , -1 = "-1"

    if(pairs_triplets):
        pairs_tri = np.array(p_t,dtype=np.str)
        pairs_tri = ''.join(pairs_tri)
        f4 = np.zeros(64)
        f5 = np.zeros(512)
        for  i in range(8):
            for j in range(8):
                check = str(i)+str(j)
                f4[i*8+j] = pairs_tri.count(check)
                for k in range(8):
                    check = str(i)+str(j)+str(k)
                    f5[i*64+j*8+k] = pairs_tri.count(check)




    f1,_ = np.histogram(global_chain_code,8)
    f2,_ = np.histogram(first_difference,8)
    f3,_ = np.histogram(second_difference,8)
    f2 = np.append(f2[:5],f2[6:])
    f1 = f1/np.sum(f1)
    f2 = f2/np.sum(f2)
    f3 = f3/np.sum(f3)
    
    if pairs_triplets:
        f4 = f4/f4.sum()
        f5 = f5/f5.sum()
        return f1,f2,f3,f4,f5
    
    
    return f1,f2,f3

In [104]:
def Chain_Codes_Feature_Extractor(image_name):
    img = io.imread(image_name)
    st = time.time()
    windows, clusters, clusteredWindows = getClusteredWindows(img,13,10)
    en = time.time()
    print("kesho ",(en-st)*1000)
    st = time.time()
    img = io.imread(image_name)
    edged = cv2.Canny(img, 30, 200)
    contours, hierarchy = cv2.findContours(edged,  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    
    
    #img = np.ones(img.shape)*255
    #cv2.drawContours(img, contours, -1, (30, 255, 30), 1) 
   
    F1,F2,F3,F4,F5 = Chain_Codes_Counting_VEC(contours,True)
   
        
    
    F7 = np.array([])
    F8 = np.array([])
    F9 = np.array([])
    
    for i in range(clusteredWindows.shape[0]):
        edged = cv2.Canny(clusteredWindows[i].astype(np.uint8), 30, 200)
        contours, hierarchy = cv2.findContours(edged,  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        F7_local , F8_local , F9_local  = Chain_Codes_Counting_VEC(contours,False)
      
            
        F7 = np.append(F7_local,F7)
        F8 = np.append(F8_local,F8)
        F9 = np.append(F9_local,F9)
    
    en = time.time()
    print((en-st)*1000)
    
    
    return F1,F2,F3,F4,F5,F7,F8,F9
    
    




   

    
    

In [106]:
F1,F2,F3,F4,F5,F7,F8,F9 = Chain_Codes_Feature_Extractor('TestCases/0.png')

kesho  15544.873714447021
639.5964622497559
